In [2]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import pickle
from IPython.display import clear_output

# `Clean dataset`

In [3]:
data = pd.read_csv('sales.csv')
data.shape

(640840, 10)

In [4]:
data.columns = data.columns.str.lower()
data.columns

Index(['unnamed: 0', 'store_id', 'day_of_week', 'date', 'nb_customers_on_day',
       'open', 'promotion', 'state_holiday', 'school_holiday', 'sales'],
      dtype='object')

In [5]:
data.head()

,unnamed: 0,store_id,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales
0,425390,366,4,2013-04-18,517,1,0,0,0,4422
1,291687,394,6,2015-04-11,694,1,0,0,0,8297
2,411278,807,4,2013-08-29,970,1,1,0,0,9729
3,664714,802,2,2013-05-28,473,1,1,0,0,6513
4,540835,726,4,2013-10-10,1068,1,1,0,0,10882


In [6]:
data = data.drop('unnamed: 0', axis=1)
data.shape

(640840, 9)

In [7]:
data.head()

,store_id,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales
0,366,4,2013-04-18,517,1,0,0,0,4422
1,394,6,2015-04-11,694,1,0,0,0,8297
2,807,4,2013-08-29,970,1,1,0,0,9729
3,802,2,2013-05-28,473,1,1,0,0,6513
4,726,4,2013-10-10,1068,1,1,0,0,10882


In [8]:
data.dtypes

store_id                int64
day_of_week             int64
date                   object
nb_customers_on_day     int64
open                    int64
promotion               int64
state_holiday          object
school_holiday          int64
sales                   int64
dtype: object

In [9]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

data.drop('date', axis=1, inplace=True)

In [10]:
data['state_holiday'] = data['state_holiday'].map({'0': 0, 'a': 1, 'b': 2, 'c': 3})

In [11]:
data.head()

,store_id,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales,year,month,day
0,366,4,517,1,0,0,0,4422,2013,4,18
1,394,6,694,1,0,0,0,8297,2015,4,11
2,807,4,970,1,1,0,0,9729,2013,8,29
3,802,2,473,1,1,0,0,6513,2013,5,28
4,726,4,1068,1,1,0,0,10882,2013,10,10


In [12]:
data.dtypes

store_id               int64
day_of_week            int64
nb_customers_on_day    int64
open                   int64
promotion              int64
state_holiday          int64
school_holiday         int64
sales                  int64
year                   int64
month                  int64
day                    int64
dtype: object

# `Filter dataset`

In [13]:
data_open_days = data[data['open'] == 1].copy()

data_open_days.drop('open', axis=1, inplace=True)

data_open_days.shape

(532016, 10)

In [14]:
data_without_store_ID = data.drop('store_id', axis=1)
data_without_store_ID.shape

(640840, 10)

In [15]:
data_open_days_without_store_ID = data_open_days.drop('store_id', axis=1)
data_open_days_without_store_ID.shape

(532016, 9)

# `Check correlations`

In [ ]:
def display_heatmap(dataframe):
    corr=dataframe.corr()

    mask=np.triu(np.ones_like(corr, dtype=bool))     # generate a mask for the upper triangle

    f, ax=plt.subplots(figsize=(11, 9))                 # set up the matplotlib figure

    cmap=sns.diverging_palette(220, 10, as_cmap=True)   # generate a custom diverging colormap

    sns.heatmap(corr, mask=mask, cmap=cmap,             # draw the heatmap with the mask and correct aspect ratio
                vmax=.3, center=0, square=True,
                linewidths=.5, cbar_kws={"shrink": .5})

## `Not filtered - without store_ID`

In [ ]:
display_heatmap(data_without_store_ID)

## `Not filtered - with store_ID`

In [ ]:
display_heatmap(data)

## `Filtered - without store_ID`

In [ ]:
display_heatmap(data_open_days_without_store_ID)

## `Filtered - with store_ID`

In [ ]:
display_heatmap(data_open_days)

# `Test algorithms`

In [16]:
def compare_models(models, dataframe):
    fitted_models = []

    model_list = []
    r2_list = []
    mse_list = []
    rmse_list = []
    mae_list = []

    for model in models:

        print(f"====== {model} ======")

        full_df = dataframe.sample(frac=1, random_state=10)

        y = full_df['sales']
        X = full_df.drop('sales', axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

        # Scaling data = X_train
        min_max_scaler = MinMaxScaler().fit(X_train)
        X_train_normalized = min_max_scaler.transform(X_train)
        X_train_normalized = pd.DataFrame(X_train_normalized)

        # Scaling data = X_test
        X_test_normalized = min_max_scaler.transform(X_test)
        X_test_normalized = pd.DataFrame(X_test_normalized)

        model.fit(X_train_normalized, y_train)

        fitted_models.append(model)

        # Make predictions on the test data
        y_pred = model.predict(X_test_normalized)

        # R2 validation
        r2 = r2_score(y_test, y_pred)
        print("R2:", r2)

        # MSE validation
        mse=mean_squared_error(y_test, y_pred)
        print("MSE:", mse)

        # RMSE validation
        rmse = np.sqrt(mse)
        print("RMSE:", rmse)

        # MAE validation
        mae=mean_absolute_error(y_test, y_pred)
        print("MAE:", mae)

        model_list.append(model)
        r2_list.append(r2)
        mse_list.append(mse)
        rmse_list.append(rmse)
        mae_list.append(mae)


    summary = {
        'Algorithm': model_list,
        'R2': r2_list,
        'MSE': mse_list,
        'RMSE': rmse_list,
        'MAE': mae_list
    }
    summary = pd.DataFrame(summary)

    return summary, fitted_models, min_max_scaler

In [17]:
def display_ordered_rmse(dataframe_summary):
    clear_output()
    display(dataframe_summary.sort_values(by='RMSE'))

In [18]:
models_to_test = [
    LinearRegression(n_jobs=-1),
    Lasso(random_state=10),
    Ridge(random_state=10),
    ElasticNet(random_state=10),
    XGBRegressor(),
    LGBMRegressor(n_jobs=-1, random_state=10),
    DecisionTreeRegressor()
]

models_to_test_slow = [
    KNeighborsRegressor(n_jobs=-1),
    MLPRegressor(random_state=10),
    RandomForestRegressor(n_jobs=-1, random_state=10)
]

## `Not filtered - without store_ID`

In [ ]:
data_without_store_ID_summary, data_without_store_ID_models, data_without_store_ID_min_max_scaler = compare_models(models=models_to_test, dataframe=data_without_store_ID)
display_ordered_rmse(data_without_store_ID_summary)

In [ ]:
# ====== LGBMRegressor(random_state=10) ======
# R2: 0.8717880480454334
# MSE: 1920020.1403067634
# RMSE: 1385.6479135432505
# MAE: 934.6317351470149

In [ ]:
data_without_store_ID_slow_summary, data_without_store_ID_models_slow = compare_models(models=models_to_test_slow, dataframe=data_without_store_ID)

In [ ]:
# ====== MLPRegressor(random_state=10) ======
# R2: 0.8604318711437954
# MSE: 2090082.977941105
# RMSE: 1445.711927716274
# MAE: 980.7829302460166

## `Not filtered - with store_ID`

In [ ]:
data_with_store_ID_summary, data_with_store_ID_models, data_with_store_ID_min_max_scaler = compare_models(models=models_to_test, dataframe=data)
display_ordered_rmse(data_with_store_ID_summary)

In [ ]:
# ====== XGBRegressor() ======
# R2: 0.9494461028054196
# MSE: 757062.8112657383
# RMSE: 870.0935646617198
# MAE: 628.8746932658818

In [ ]:
data_with_store_ID_slow_summary, data_with_store_ID_models_slow = compare_models(models=models_to_test_slow, dataframe=data)

In [ ]:
# ====== RandomForestRegressor(n_jobs=-1, random_state=10) ======
# R2: 0.9228395853784788
# MSE: 1155504.9888035383
# RMSE: 1074.9441793895803
# MAE: 724.3297227857188

## `Filtered - without store_ID`

In [ ]:
data_open_days_without_store_ID_summary, data_open_days_without_store_ID_models, data_open_days_without_store_ID_min_max_scaler = compare_models(models=models_to_test, dataframe=data_open_days_without_store_ID)
display_ordered_rmse(data_open_days_without_store_ID_summary)

In [ ]:
# ====== LinearRegression(n_jobs=-1) ======
# R2: 0.7115516521122665
# MSE: 2807809.682227139
# RMSE: 1675.6520170450483
# MAE: 1206.1741847849569

In [ ]:
data_open_days_without_store_ID_slow_summary, data_open_days_without_store_ID_models_slow = compare_models(models=models_to_test_slow, dataframe=data_open_days_without_store_ID)

In [ ]:
# ====== MLPRegressor(random_state=10) ======
# R2: 0.7423804028651348
# MSE: 2507716.9082911913
# RMSE: 1583.5772504968588
# MAE: 1168.3293276782179

## `Filtered - with store_ID`

In [ ]:
data_open_days_with_store_ID_summary, data_open_days_with_store_ID_models, data_open_days_with_store_ID_min_max_scaler = compare_models(models=models_to_test, dataframe=data_open_days)
display_ordered_rmse(data_open_days_with_store_ID_summary)

In [ ]:
# ====== XGBRegressor() ======
# R2: 0.9151376668371063
# MSE: 826065.6802371338
# RMSE: 908.8815545697546
# MAE: 689.5817966818616

In [ ]:
data_open_days_with_store_ID_slow_summary, data_open_days_with_store_ID_models_slow = compare_models(models=models_to_test_slow, dataframe=data_open_days)

# `For now : Not filtered - with store_ID -> data_with_store_ID_models, XGBRegressor()`

# `Try find best parameters`

In [ ]:
def find_best_parameters(model, param_grid, nb_cross_validations):
    grid_search_decision_tree_classifier = GridSearchCV(estimator = model, param_grid = param_grid, cv = nb_cross_validations, n_jobs=-1)

    full_df = data.sample(frac=1, random_state=10)

    y = full_df['sales']
    X = full_df.drop('sales', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

    # Scaling data = X_train
    X_train_transformer = MinMaxScaler().fit(X_train)
    X_train_normalized = X_train_transformer.transform(X_train)
    X_train_normalized = pd.DataFrame(X_train_normalized)

    grid_search_decision_tree_classifier.fit(X_train_normalized, y_train)

    print(f"Best parameters: {grid_search_decision_tree_classifier.best_params_}")
    print(f"Best score: {grid_search_decision_tree_classifier.best_score_}")

    return grid_search_decision_tree_classifier

In [ ]:
grid = {
    'booster': ['gbtree', 'gblinear', 'dart', 'gbtree', 'dart'],
    'validate_parameters': [True, False]
}

In [ ]:
best_parameters_XGBRegressor_5_cv = find_best_parameters(model=XGBRegressor(), param_grid=grid, nb_cross_validations=5)

In [ ]:
best_parameters_XGBRegressor_10_cv = find_best_parameters(model=XGBRegressor(), param_grid=grid, nb_cross_validations=10)

# `Best parameters for XGBRegressor seem to be : 'booster': 'gbtree', 'validate_parameters': True`

In [ ]:
XGBRegressor_with_best_parameters_summary, XGBRegressor_with_best_parameters_models = compare_models(models=[XGBRegressor(booster='gbtree', validate_parameters=True, n_jobs=-1)], dataframe=data)

In [ ]:
# ====== XGBRegressor(base_score=None, booster='gbtree', colsample_bylevel=None,
#              colsample_bynode=None, colsample_bytree=None,
#              enable_categorical=False, gamma=None, gpu_id=None,
#              importance_type=None, interaction_constraints=None,
#              learning_rate=None, max_delta_step=None, max_depth=None,
#              min_child_weight=None, missing=nan, monotone_constraints=None,
#              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
#              predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
#              scale_pos_weight=None, subsample=None, tree_method=None,
#              validate_parameters=True, verbosity=None) ======
# /opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
#   elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

# R2: 0.9494461028054196
# MSE: 757062.8112657383
# RMSE: 870.0935646617198
# MAE: 628.8746932658818

In [ ]:
XGBRegressor_default_parameters_summary, XGBRegressor_default_parameters_models = compare_models(models=[XGBRegressor(n_jobs=-1)], dataframe=data)

In [ ]:
# ====== XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
#              colsample_bynode=None, colsample_bytree=None,
#              enable_categorical=False, gamma=None, gpu_id=None,
#              importance_type=None, interaction_constraints=None,
#              learning_rate=None, max_delta_step=None, max_depth=None,
#              min_child_weight=None, missing=nan, monotone_constraints=None,
#              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
#              predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
#              scale_pos_weight=None, subsample=None, tree_method=None,
#              validate_parameters=None, verbosity=None) ======

# R2: 0.9494461028054196
# MSE: 757062.8112657383
# RMSE: 870.0935646617198
# MAE: 628.8746932658818

# `Save XGBRegressor_default_parameters_model`

In [ ]:
with open(f'models/XGBRegressor_sales_model.pkl', 'wb') as f:
    pickle.dump(XGBRegressor_default_parameters_models[0], f)

# `Load XGBRegressor_default_parameters_model`

In [ ]:
with open(f'models/XGBRegressor_sales_model.pkl', "rb") as f:
    sales_model = pickle.load(f)

In [ ]:
data['day_of_week'].unique()

# `Try create models per unique values of specific feature`

In [19]:
def create_models_per_feature(feature_name, dataframe):
    unique_values = dataframe[feature_name].unique()

    # Split dataframe into dataframes per day_of_week
    filtered_df_list = [
        {
            f'{feature_name}': value, 
            'dataframe': dataframe[dataframe[feature_name] == value].drop(feature_name, axis=1)
        }
                                                    for value in unique_values
    ]

    models_per_feature_results = [
        {
            f'{feature_name}': obj[feature_name],
            'summary_and_fitted_model': compare_models(models=[XGBRegressor(n_jobs=-1)], dataframe=obj['dataframe'])
        }
        for obj in filtered_df_list
    ]

    # Create results dataframe
    models_per_feature_summaries = [result['summary_and_fitted_model'][0].drop('Algorithm', axis=1).assign(feature_value = result[feature_name]) for result in models_per_feature_results]
    models_per_feature_comparison = pd.concat(models_per_feature_summaries, axis=0).sort_values(by='RMSE').reset_index().drop('index', axis=1)
    
    clear_output()
    display(models_per_feature_comparison)

    models = [
        {
            'feature_value': result[feature_name],
            'model': result['summary_and_fitted_model'][1],
            'min_max_scaler': result['summary_and_fitted_model'][2]
        }
            for result in models_per_feature_results
    ]

    return models

## `Not filtered`

### `day_of_week`

In [20]:
day_of_week_models = create_models_per_feature('day_of_week', data)

,R2,MSE,RMSE,MAE,feature_value
0,0.996390,10142.094842,100.707968,11.121698,7
1,0.947799,439136.213817,662.673535,486.736620,6
2,0.950888,512042.095957,715.571168,527.808966,4
3,0.934467,557354.409070,746.561725,542.751524,3
4,0.938844,582190.793056,763.014281,552.531702,2
5,0.938092,603189.153370,776.652531,567.940734,5
6,0.945989,849694.081415,921.788523,661.846415,1


In [ ]:
array = [100.707968, 662.673535, 715.437158, 746.561725, 763.014281, 780.234802, 921.788523]
sum(array) / len(array)

### `day`

In [ ]:
create_models_per_feature('day', data)

### `month`

In [ ]:
create_models_per_feature('month', data)

### `year`

In [ ]:
create_models_per_feature('year', data)

### `open`

In [ ]:
create_models_per_feature('open', data)

### `promotion`

In [ ]:
create_models_per_feature('promotion', data)

In [ ]:
array = [601.682359, 853.785553]
sum(array) / len(array)

### `state_holiday`

In [21]:
state_holiday_models = create_models_per_feature('state_holiday', data)

,R2,MSE,RMSE,MAE,feature_value
0,0.971664,47656.891899,218.304585,16.213130,3
1,0.987031,58129.355901,241.100303,29.515045,1
2,0.976903,60040.309570,245.031242,27.247376,2
3,0.959989,572472.726326,756.619274,528.465144,0


In [ ]:
array = [218.304585, 241.100303, 245.031242, 756.619274]
sum(array) / len(array)

### `school_holiday`

In [ ]:
create_models_per_feature('school_holiday', data)

### `store_ID`

In [ ]:
store_ID_models = create_models_per_feature('store_ID', data)

In [ ]:
data.nunique()

## `Filtered`

### `day_of_week`

In [ ]:
create_models_per_feature('day_of_week', data_open_days)

### `day`

In [ ]:
create_models_per_feature('day', data_open_days)

### `promotion`

In [ ]:
create_models_per_feature('promotion', data_open_days)

### `state_holiday`

In [ ]:
create_models_per_feature('state_holiday', data_open_days)

# `Save models per day_of_week`

In [ ]:
def save_model(model):
    with open(f"models/XGBRegressor_sales_model_day{model['feature_value']}.pkl", 'wb') as f:
        pickle.dump(model['model'], f)

[save_model(model) for model in day_of_week_models]

# `Load models per day_of_week`

In [ ]:
with open(f'models/XGBRegressor_sales_model_day1.pkl', "rb") as f:
    XGBRegressor_sales_model_day1_model = pickle.load(f)

with open(f'models/XGBRegressor_sales_model_day2.pkl', "rb") as f:
    XGBRegressor_sales_model_day2_model = pickle.load(f)

with open(f'models/XGBRegressor_sales_model_day3.pkl', "rb") as f:
    XGBRegressor_sales_model_day3_model = pickle.load(f)

with open(f'models/XGBRegressor_sales_model_day4.pkl', "rb") as f:
    XGBRegressor_sales_model_day4_model = pickle.load(f)

with open(f'models/XGBRegressor_sales_model_day5.pkl', "rb") as f:
    XGBRegressor_sales_model_day5_model = pickle.load(f)

with open(f'models/XGBRegressor_sales_model_day6.pkl', "rb") as f:
    XGBRegressor_sales_model_day6_model = pickle.load(f)

with open(f'models/XGBRegressor_sales_model_day7.pkl', "rb") as f:
    XGBRegressor_sales_model_day7_model = pickle.load(f)

# `Predict sales`

In [22]:
def predict_data_feature_value(prediction_dataframe, model, min_max_scaler, feature_name, feature_value):
    filtered_df = prediction_dataframe[prediction_dataframe[feature_name] == feature_value]
    filtered_df_normalized = min_max_scaler.transform(filtered_df.drop(['true_index', feature_name], axis=1))
    predictions = model.predict(filtered_df_normalized)
    df = pd.concat([filtered_df.reset_index(), pd.DataFrame(predictions, columns=['sales'])], axis=1)

    return df

def predict_data(model_dataframe, prediction_dataframe, feature_name):
    models = create_models_per_feature(feature_name, model_dataframe)
    feature_unique_values = sorted(prediction_dataframe[feature_name].unique())
    predictions_list = [predict_data_feature_value(prediction_dataframe, models['model'][0], models['min_max_scaler'], feature_name, feature_value) for feature_value, models in zip(feature_unique_values, models)]
    return pd.concat(predictions_list, axis=0)

In [23]:
def get_cleaned_predicted_data(dataframe):
    incorrect_predicted_data = dataframe[dataframe['sales'] < 0]
    incorrect_predicted_data['sales'] = 0

    print(f"Shape incorrect_predictions: {incorrect_predicted_data.shape}")

    correct_predicted_data = dataframe[(dataframe['sales'] >= 0)]
    correct_predicted_data.shape

    print(f"Shape correct_predictions: {correct_predicted_data.shape}")

    cleaned_predictions = pd.concat([incorrect_predicted_data, correct_predicted_data], axis=0)

    print(f"Shape cleaned_predictions: {cleaned_predictions.shape}")

    return cleaned_predictions

In [24]:
def transform_and_save_prediction_dataframe(dataframe, rounded_type):
    # Remove useless columns
    dataframe = dataframe.rename(columns={
        'true_index' : 'True_index',
        'sales': 'Sales'
    })
    # Round values
    if rounded_type == 'round':
        dataframe['Sales'] = dataframe['Sales'].round()
        dataframe['Sales'] = dataframe['Sales'].astype(int)
    elif rounded_type == 'floor':
        dataframe['Sales'] = dataframe['Sales'].apply(np.floor)
        dataframe['Sales'] = dataframe['Sales'].astype(int)

    dataframe = dataframe[['True_index', 'Sales']].sort_values(by='True_index')
    dataframe.to_csv(f'predicted_data_{rounded_type}.csv', index=False)

    return dataframe

## `Map state_holiday`

In [25]:
test = pd.read_csv('validation_for_students.csv')
test.head()

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday
0,7,764,4,2013-12-26,0,0,0,c,1
1,19,22,3,2013-05-22,449,1,0,0,1
2,31,1087,6,2013-06-29,622,1,0,0,0
3,45,139,6,2013-08-17,314,1,0,0,0
4,56,568,1,2014-04-07,356,1,0,0,0


In [26]:
validation_data = pd.read_csv('validation_for_students.csv')

validation_data.columns = validation_data.columns.str.lower()

validation_data['date'] = pd.to_datetime(validation_data['date'])
validation_data['year'] = validation_data['date'].dt.year
validation_data['month'] = validation_data['date'].dt.month
validation_data['day'] = validation_data['date'].dt.day

validation_data.drop('date', axis=1, inplace=True)

validation_data['state_holiday'] = validation_data['state_holiday'].map({'0': 0, 'a': 1, 'b': 2, 'c': 3})

print(f"Shape: {validation_data.shape}")
validation_data.head()

Shape: (71205, 11)


,true_index,store_id,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,year,month,day
0,7,764,4,0,0,0,3,1,2013,12,26
1,19,22,3,449,1,0,0,1,2013,5,22
2,31,1087,6,622,1,0,0,0,2013,6,29
3,45,139,6,314,1,0,0,0,2013,8,17
4,56,568,1,356,1,0,0,0,2014,4,7


In [27]:
predicted_data = predict_data(data, validation_data, 'state_holiday')
predicted_data.shape

,R2,MSE,RMSE,MAE,feature_value
0,0.971664,47656.891899,218.304585,16.213130,3
1,0.987031,58129.355901,241.100303,29.515045,1
2,0.976903,60040.309570,245.031242,27.247376,2
3,0.959989,572472.726326,756.619274,528.465144,0


(71205, 13)

In [28]:
array = [218.304585, 241.100303, 245.031242, 756.619274]
np.mean(array)

365.26385100000005

In [29]:
cleaned_predicted_data = get_cleaned_predicted_data(predicted_data)
cleaned_predicted_data.head()

Shape incorrect_predictions: (4912, 13)
Shape correct_predictions: (66293, 13)
Shape cleaned_predictions: (71205, 13)


/var/folders/x6/07vbnp4935lcxx8dmdvxd3700000gn/T/ipykernel_11100/3185454249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_predicted_data['sales'] = 0


,index,true_index,store_id,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,year,month,day,sales
13,14,105,581,7,0,0,0,0,0,2015,7,5,0.0
37,39,281,583,7,0,0,0,0,0,2014,3,23,0.0
41,43,327,264,7,0,0,0,0,0,2013,10,20,0.0
47,49,392,776,7,0,0,0,0,0,2015,1,18,0.0
51,53,466,1065,7,0,0,0,0,0,2013,3,17,0.0


In [30]:
cleaned_predicted_data = transform_and_save_prediction_dataframe(cleaned_predicted_data, 'floor')
cleaned_predicted_data.head()

,True_index,Sales
0,7,0
0,19,3721
1,31,6340
2,45,3315
3,56,3468


In [31]:
cleaned_predicted_data['Sales'].mean()

5780.565746787444

In [32]:
max_csv = pd.read_csv('predicted_data_max.csv')
max_csv['0'].mean()

5770.694940110116

## `OneHotEncode state_holiday`

In [33]:
validation_data = pd.read_csv('validation_for_students.csv')

validation_data.columns = validation_data.columns.str.lower()

validation_data['date'] = pd.to_datetime(validation_data['date'])
validation_data['year'] = validation_data['date'].dt.year
validation_data['month'] = validation_data['date'].dt.month
validation_data['day'] = validation_data['date'].dt.day

validation_data.drop('date', axis=1, inplace=True)

validation_data = pd.get_dummies(validation_data, columns=['state_holiday'], prefix='state_holiday')

print(f"Shape: {validation_data.shape}")
validation_data.head()

Shape: (71205, 14)


,true_index,store_id,day_of_week,nb_customers_on_day,open,promotion,school_holiday,year,month,day,state_holiday_0,state_holiday_a,state_holiday_b,state_holiday_c
0,7,764,4,0,0,0,1,2013,12,26,0,0,0,1
1,19,22,3,449,1,0,1,2013,5,22,1,0,0,0
2,31,1087,6,622,1,0,0,2013,6,29,1,0,0,0
3,45,139,6,314,1,0,0,2013,8,17,1,0,0,0
4,56,568,1,356,1,0,0,2014,4,7,1,0,0,0


In [34]:
data_one_hot_encoded = pd.read_csv('sales.csv')

data_one_hot_encoded.columns = data_one_hot_encoded.columns.str.lower()

data_one_hot_encoded = data_one_hot_encoded.drop('unnamed: 0', axis=1)

data_one_hot_encoded['date'] = pd.to_datetime(data_one_hot_encoded['date'])
data_one_hot_encoded['year'] = data_one_hot_encoded['date'].dt.year
data_one_hot_encoded['month'] = data_one_hot_encoded['date'].dt.month
data_one_hot_encoded['day'] = data_one_hot_encoded['date'].dt.day

data_one_hot_encoded.drop('date', axis=1, inplace=True)

data_one_hot_encoded = pd.get_dummies(data_one_hot_encoded, columns=['state_holiday'], prefix='state_holiday')

data_one_hot_encoded.head()

,store_id,day_of_week,nb_customers_on_day,open,promotion,school_holiday,sales,year,month,day,state_holiday_0,state_holiday_a,state_holiday_b,state_holiday_c
0,366,4,517,1,0,0,4422,2013,4,18,1,0,0,0
1,394,6,694,1,0,0,8297,2015,4,11,1,0,0,0
2,807,4,970,1,1,0,9729,2013,8,29,1,0,0,0
3,802,2,473,1,1,0,6513,2013,5,28,1,0,0,0
4,726,4,1068,1,1,0,10882,2013,10,10,1,0,0,0


In [35]:
state_holiday_0_models = create_models_per_feature('state_holiday_0', data_one_hot_encoded)
state_holiday_a_models = create_models_per_feature('state_holiday_a', data_one_hot_encoded)
state_holiday_b_models = create_models_per_feature('state_holiday_b', data_one_hot_encoded)
state_holiday_c_models = create_models_per_feature('state_holiday_c', data_one_hot_encoded)

,R2,MSE,RMSE,MAE,feature_value
0,0.971664,47656.891899,218.304585,16.213130,1
1,0.961237,572571.932253,756.684830,522.972155,0


In [36]:
array = [756.619274, 241.100303, 245.031242, 218.304585]
np.mean(array)

365.26385100000005

In [37]:
feature_name_list = ['state_holiday_0', 'state_holiday_a', 'state_holiday_b', 'state_holiday_c']
state_holiday_models = [state_holiday_0_models, state_holiday_a_models, state_holiday_b_models, state_holiday_c_models]

def predict_state_holiday_data(feature_name, models):
    model = [model for model in models if model['feature_value'] == 1][0]
    filtered_df = validation_data[validation_data[feature_name] == 1]
    filtered_df_normalized = model['min_max_scaler'].transform(filtered_df.drop(['true_index', feature_name], axis=1))
    predictions = model['model'][0].predict(filtered_df_normalized)
    return pd.concat([filtered_df.reset_index(), pd.DataFrame(predictions, columns=['sales'])], axis=1)


predictions_list = [predict_state_holiday_data(feature_name, models) for feature_name, models in zip(feature_name_list, state_holiday_models)]
one_hot_encoded_predicted_data = pd.concat(predictions_list, axis=0)

one_hot_encoded_predicted_data.shape

(71205, 16)

In [38]:
one_hot_encoded_cleaned_predicted_data = get_cleaned_predicted_data(one_hot_encoded_predicted_data)
one_hot_encoded_cleaned_predicted_data.head()

Shape incorrect_predictions: (4912, 16)
Shape correct_predictions: (66293, 16)
Shape cleaned_predictions: (71205, 16)


/var/folders/x6/07vbnp4935lcxx8dmdvxd3700000gn/T/ipykernel_11100/3185454249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_predicted_data['sales'] = 0


,index,true_index,store_id,day_of_week,nb_customers_on_day,open,promotion,school_holiday,year,month,day,state_holiday_0,state_holiday_a,state_holiday_b,state_holiday_c,sales
13,14,105,581,7,0,0,0,0,2015,7,5,1,0,0,0,0.0
37,39,281,583,7,0,0,0,0,2014,3,23,1,0,0,0,0.0
41,43,327,264,7,0,0,0,0,2013,10,20,1,0,0,0,0.0
47,49,392,776,7,0,0,0,0,2015,1,18,1,0,0,0,0.0
51,53,466,1065,7,0,0,0,0,2013,3,17,1,0,0,0,0.0


In [39]:
one_hot_encoded_cleaned_predicted_data = transform_and_save_prediction_dataframe(one_hot_encoded_cleaned_predicted_data, 'floor')
one_hot_encoded_cleaned_predicted_data.head()

,True_index,Sales
0,7,0
0,19,3721
1,31,6340
2,45,3315
3,56,3468


In [40]:
one_hot_encoded_cleaned_predicted_data['Sales'].mean()

5780.539021136156